In [1]:
# this script has a function which takes argumants file path, name pattern as inputs and returns annotation dictionary
# this function is used to extract sub entities as per 24/08/2021

In [7]:
import json
import glob
import os
import re
import pandas as pd

In [8]:
# Folder_name = path to the folder which contains json files
#foldername = "D:\FYP\Annotation JSON\Inception\JSON fils Suwani"


In [9]:
# getting the full json file list
#file_list = glob.glob(os.path.join(os.getcwd(), foldername, "*.json"))

In [10]:
# for suwani
#namepattern = 'S\_\d\d?\d?\.json'

In [11]:
# initializing a dictionary to put all the content
annotations = {}
TRAIN_DATA = []
notworked = []

In [12]:
def extractAnnotation(foldername,namepattern,Name_Entity,entword):
    """
    Argumants example
    foldername = "D:\FYP\Annotation JSON\Inception\JSON fils Suwani"
    namepattern = 'S\_\d\d?\d?\.json'
    Name_Entity: The name of the dictionary where all the entities are saved in the json files 
    """
    file_list = glob.glob(os.path.join(os.getcwd(), foldername, "*.json"))
    #for i in range(500):
    for i in range(len(file_list)):
        print(i)
        print(file_list[i])
        name = re.findall(namepattern, file_list[i])
        print(name[0])
        with open(file_list[i],encoding='utf-8') as data_file:
            data = json.load(data_file)
        # Getting the entire text
        fulltext = data['_referenced_fss']['1']['sofaString']
        # Replacing 
        sentences_all =fulltext.replace("â€¢","\n")
        ## Replacing \r\n with two spaces and \n with one space
        sentences_all = sentences_all.replace('\r\n', '  ').replace('\n',' ')
        # Getting the sentence boundaries
        Sentence = data['_views']['_InitialView']['Sentence']
        # Set first sentence starting position 0
        Sentence[0]['begin'] = 0
        sentences_list = []
        # Making sentence list
        for i in range(len(Sentence)):
            sent = sentences_all[Sentence[i]['begin']:Sentence[i]['end']]
            sentences_list.append(sent)
        try:
            # Extract entity start/ end positions and names
            ent_loc = data['_views']['_InitialView'][Name_Entity]

            # if 'begin' in not present create new key called begin and assign 0
            for i in range(len(ent_loc)):
                #print(ent_loc[i])
                if 'begin' not in ent_loc[i]:
                    ent_loc[i]['begin'] = 0
                   # print(ent_loc[i])
             # if 'Entities' or 'Value' in not present create new key called Entities and assign "Not annotated"
            #entword = 'Entities'
            for i in range(len(ent_loc)):
            #print(ent_loc[i])
                if entword not in ent_loc[i]:
                    ent_loc[i][entword] = 'Not annotated'
               # print(ent_loc[i])
            
            delentities = ['job_title','address','age_limit','years_experienced','age_limit','education','years_exp','adrs']
            ent_loc_final = []
            for ent in ent_loc:
                #print(ent['Entities'])
                if ent[entword] not in delentities:
                    ent_loc_final.append(ent)
                else:
                    print(ent)


            #TRAIN_DATA = []
            ent_list = []
            for sl in range(len(Sentence)):
                #print(".....................")
                #print(Sentence[sl])
                ent_list_sen = []
                #print(".....................")
                for el in range(len(ent_loc_final)):
                    if 'begin' not in ent_loc_final[el]:
                        ent_loc_final[el]['begin'] = 0
                    #print(ent_loc[el])
                    if(ent_loc_final[el]['begin'] >= Sentence[sl]['begin'] and ent_loc_final[el]['end'] <= Sentence[sl]['end']):
                        ent_list_sen.append([(ent_loc_final[el]['begin']-Sentence[sl]['begin']),(ent_loc_final[el]['end']-Sentence[sl]['begin']),ent_loc_final[el][entword]])

                ent_list.append(ent_list_sen)
                ## Create blank dictionary
                ent_dic = {}
                ## Fill value to the dictionary
                ## Only append where the is an entitiy 
                if len(ent_list[-1]) != 0:
                    ent_dic['entities'] = ent_list[-1]
                    ## Prepare final training data
                    TRAIN_DATA.append([sentences_list[sl],ent_dic])
            #name = re.findall(namepattern, file_list[i])
            #print(name)
            annotations[name[0]] = TRAIN_DATA
        except KeyError:
            print("Didnt work for {}".format(name[0]))
            notworked.append(name[0])

In [13]:
extractlist = [extractAnnotation("D:\FYP\Annotation JSON\Inception\JSON fils Suwani",'S\_\d\d?\d?\.json','Name_Entity','Entities'),
extractAnnotation("D:\FYP\Annotation JSON\Inception\JASON files Diluni",'D\_\d\d?\d?\.json','NamedEntity','value'),
extractAnnotation("D:\FYP\Annotation JSON\Inception\Jason Himali",'admin\(\d\d?\d?\)\.json','NamedEntity','value'),
extractAnnotation("D:\FYP\Annotation JSON\Inception\Inception_Harini",'HD\_\d\d?\d?\.json','NER','Entities')]

for i in extractlist:
    i


0
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_1.json
S_1.json
{'sofa': 1, 'begin': 640, 'end': 685, 'Entities': 'address'}
{'sofa': 1, 'begin': 698, 'end': 713, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 1305, 'end': 1320, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 3514, 'end': 3564, 'Entities': 'address'}
1
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_10.json
S_10.json
{'sofa': 1, 'begin': 260, 'end': 281, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 268, 'end': 281, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 316, 'end': 342, 'Entities': 'age_limit'}
{'sofa': 1, 'begin': 414, 'end': 433, 'Entities': 'years_exp'}
{'sofa': 1, 'begin': 740, 'end': 792, 'Entities': 'address'}
2
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_100.json
S_100.json
{'sofa': 1, 'begin': 27, 'end': 45, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 553, 'end': 568, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 1230, 'end': 1253, 'Entities': 'years_experienced'}
3
D:\FYP\Annotation JS

{'sofa': 1, 'begin': 5, 'end': 19, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 1044, 'end': 1061, 'Entities': 'age_limit'}
{'sofa': 1, 'begin': 1316, 'end': 1348, 'Entities': 'address'}
180
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_261.json
S_261.json
{'sofa': 1, 'begin': 9, 'end': 19, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 1919, 'end': 1948, 'Entities': 'age_limit'}
{'sofa': 1, 'begin': 2351, 'end': 2384, 'Entities': 'address'}
181
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_262.json
S_262.json
{'sofa': 1, 'end': 30, 'Entities': 'job_title', 'begin': 0}
{'sofa': 1, 'begin': 633, 'end': 653, 'Entities': 'job_title'}
182
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_263.json
S_263.json
{'sofa': 1, 'begin': 23, 'end': 47, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 1297, 'end': 1334, 'Entities': 'years_experienced'}
183
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_264.json
S_264.json
{'sofa': 1, 'begin': 36, 'end': 52, 'Entities': 'job_title'}
{'

{'sofa': 1, 'begin': 57, 'end': 85, 'Entities': 'job_title'}
344
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_409.json
S_409.json
{'sofa': 1, 'begin': 783, 'end': 804, 'Entities': 'years_experienced'}
{'sofa': 1, 'begin': 842, 'end': 854, 'Entities': 'age_limit'}
{'sofa': 1, 'begin': 889, 'end': 911, 'Entities': 'job_title'}
345
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_41.json
S_41.json
{'sofa': 1, 'end': 21, 'Entities': 'job_title', 'begin': 0}
{'sofa': 1, 'begin': 24, 'end': 42, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 212, 'end': 235, 'Entities': 'age_limit'}
{'sofa': 1, 'begin': 562, 'end': 593, 'Entities': 'address'}
346
D:\FYP\Annotation JSON\Inception\JSON fils Suwani\S_410.json
S_410.json
{'sofa': 1, 'begin': 457, 'end': 474, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 478, 'end': 489, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 494, 'end': 503, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 507, 'end': 526, 'Entities': 'job_title'}
{'sofa': 1, 'begin'

0
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_1.json
D_1.json
{'sofa': 1, 'begin': 20, 'end': 38, 'value': 'job_title'}
1
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_10.json
D_10.json
{'sofa': 1, 'begin': 320, 'end': 330, 'value': 'job_title'}
{'sofa': 1, 'begin': 1913, 'end': 1937, 'value': 'age_limit'}
2
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_100.json
D_100.json
{'sofa': 1, 'begin': 357, 'end': 374, 'value': 'job_title'}
{'sofa': 1, 'begin': 475, 'end': 492, 'value': 'job_title'}
{'sofa': 1, 'begin': 540, 'end': 557, 'value': 'job_title'}
{'sofa': 1, 'begin': 908, 'end': 941, 'value': 'adrs'}
3
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_101.json
D_101.json
{'sofa': 1, 'begin': 1158, 'end': 1170, 'value': 'job_title'}
{'sofa': 1, 'begin': 1495, 'end': 1505, 'value': 'job_title'}
{'sofa': 1, 'begin': 1507, 'end': 1551, 'value': 'job_title'}
{'sofa': 1, 'begin': 1917, 'end': 1936, 'value': 'job_title'}
{'sofa': 1, 'begin': 2110, 'end': 2140,

{'sofa': 1, 'begin': 67, 'end': 81, 'value': 'job_title'}
{'sofa': 1, 'begin': 421, 'end': 442, 'value': 'years_exp'}
152
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_236.json
D_236.json
{'sofa': 1, 'begin': 22, 'end': 45, 'value': 'job_title'}
{'sofa': 1, 'begin': 2384, 'end': 2403, 'value': 'years_exp'}
153
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_237.json
D_237.json
{'sofa': 1, 'begin': 30, 'end': 48, 'value': 'job_title'}
{'sofa': 1, 'begin': 451, 'end': 474, 'value': 'years_exp'}
154
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_238.json
D_238.json
{'sofa': 1, 'end': 11, 'value': 'job_title', 'begin': 0}
{'sofa': 1, 'begin': 241, 'end': 249, 'value': 'years_exp'}
155
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_239.json
D_239.json
{'sofa': 1, 'end': 15, 'value': 'job_title', 'begin': 0}
{'sofa': 1, 'begin': 280, 'end': 313, 'value': 'years_exp'}
156
D:\FYP\Annotation JSON\Inception\JASON files Diluni\D_24.json
D_24.json
{'sofa': 1, 'begin': 2

...

{'sofa': 1, 'begin': 46, 'end': 76, 'value': 'job_title'}
{'sofa': 1, 'begin': 104, 'end': 121, 'value': 'age_limit'}
248
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(324).json
admin(324).json
{'sofa': 1, 'begin': 342, 'end': 358, 'value': 'job_title'}
{'sofa': 1, 'begin': 798, 'end': 813, 'value': 'age_limit'}
249
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(325).json
admin(325).json
{'sofa': 1, 'begin': 46, 'end': 72, 'value': 'job_title'}
250
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(326).json
admin(326).json
{'sofa': 1, 'begin': 97, 'end': 120, 'value': 'job_title'}
{'sofa': 1, 'begin': 121, 'end': 130, 'value': 'job_title'}
251
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(327).json
admin(327).json
{'sofa': 1, 'begin': 30, 'end': 49, 'value': 'job_title'}
{'sofa': 1, 'begin': 30, 'end': 39, 'value': 'job_title'}
{'sofa': 1, 'begin': 758, 'end': 777, 'value': 'years_exp'}
252
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(328).json
admin(328).json

{'sofa': 1, 'begin': 341, 'end': 358, 'value': 'job_title'}
{'sofa': 1, 'begin': 451, 'end': 482, 'value': 'years_exp'}
{'sofa': 1, 'begin': 666, 'end': 678, 'value': 'age_limit'}
{'sofa': 1, 'begin': 1205, 'end': 1236, 'value': 'years_exp'}
{'sofa': 1, 'begin': 1420, 'end': 1432, 'value': 'age_limit'}
{'sofa': 1, 'begin': 1659, 'end': 1690, 'value': 'years_exp'}
{'sofa': 1, 'begin': 1874, 'end': 1886, 'value': 'age_limit'}
411
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(473).json
admin(473).json
{'sofa': 1, 'begin': 1109, 'end': 1136, 'value': 'job_title'}
{'sofa': 1, 'begin': 1109, 'end': 1126, 'value': 'job_title'}
{'sofa': 1, 'begin': 2005, 'end': 2024, 'value': 'age_limit'}
{'sofa': 1, 'begin': 2098, 'end': 2116, 'value': 'years_exp'}
{'sofa': 1, 'begin': 2814, 'end': 2868, 'value': 'adrs'}
412
D:\FYP\Annotation JSON\Inception\Jason Himali\admin(474).json
admin(474).json
{'sofa': 1, 'begin': 1109, 'end': 1136, 'value': 'job_title'}
{'sofa': 1, 'begin': 1109, 'end': 1126, '

{'sofa': 1, 'end': 18, 'Entities': 'job_title', 'begin': 0}
{'sofa': 1, 'begin': 151, 'end': 174, 'Entities': 'age_limit'}
{'sofa': 1, 'begin': 429, 'end': 460, 'Entities': 'adrs'}
77
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_169.json
HD_169.json
{'sofa': 1, 'begin': 20, 'end': 27, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 89, 'end': 108, 'Entities': 'age_limit'}
{'sofa': 1, 'begin': 220, 'end': 238, 'Entities': 'years_exp'}
78
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_17.json
HD_17.json
{'sofa': 1, 'end': 23, 'Entities': 'job_title', 'begin': 0}
{'sofa': 1, 'begin': 497, 'end': 529, 'Entities': 'years_exp'}
{'sofa': 1, 'begin': 860, 'end': 875, 'Entities': 'age_limit'}
79
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_170.json
HD_170.json
{'sofa': 1, 'begin': 85, 'end': 100, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 1556, 'end': 1572, 'Entities': 'age_limit'}
{'sofa': 1, 'begin': 3294, 'end': 3339, 'Entities': 'adrs'}
80
D:\FYP\Annotation JSON\Incep

{'sofa': 1, 'end': 24, 'Entities': 'job_title', 'begin': 0}
{'sofa': 1, 'begin': 853, 'end': 888, 'Entities': 'years_exp'}
{'sofa': 1, 'begin': 2103, 'end': 2171, 'Entities': 'adrs'}
222
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_3.json
HD_3.json
{'sofa': 1, 'end': 14, 'Entities': 'job_title', 'begin': 0}
223
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_30.json
HD_30.json
{'sofa': 1, 'end': 45, 'Entities': 'adrs', 'begin': 0}
{'sofa': 1, 'begin': 69, 'end': 77, 'Entities': 'job_title'}
224
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_300.json
HD_300.json
{'sofa': 1, 'begin': 292, 'end': 316, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 1040, 'end': 1059, 'Entities': 'years_exp'}
{'sofa': 1, 'begin': 1984, 'end': 1999, 'Entities': 'age_limit'}
225
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_301.json
HD_301.json
{'sofa': 1, 'begin': 260, 'end': 270, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 479, 'end': 519, 'Entities': 'years_exp'}
226
D:\FYP\Annot

{'sofa': 1, 'begin': 131, 'end': 159, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 2735, 'end': 2844, 'Entities': 'adrs'}
416
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_474.json
HD_474.json
{'sofa': 1, 'begin': 1366, 'end': 1406, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 2855, 'end': 2878, 'Entities': 'years_exp'}
417
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_475.json
HD_475.json
{'sofa': 1, 'begin': 44, 'end': 71, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 2243, 'end': 2277, 'Entities': 'years_exp'}
418
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_476.json
HD_476.json
{'sofa': 1, 'begin': 19, 'end': 47, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 137, 'end': 159, 'Entities': 'years_exp'}
419
D:\FYP\Annotation JSON\Inception\Inception_Harini\HD_477.json
HD_477.json
{'sofa': 1, 'begin': 468, 'end': 490, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 492, 'end': 511, 'Entities': 'job_title'}
{'sofa': 1, 'begin': 513, 'end': 533, 'Entities': 'job_titl

In [15]:
notworked

[]

In [ ]:
annotations

In [6]:
TRAIN_DATA

NameError: name 'TRAIN_DATA' is not defined

In [11]:
import json

with open('AnnotatedDataSubEntities','w') as f:
    json.dump(TRAIN_DATA,f)

In [86]:
import pickle

with open('TrainingData', 'wb') as fp:
    pickle.dump(TRAIN_DATA, fp)

In [54]:
with open ('TrainingData', 'rb') as fp:
    itemlist = pickle.load(fp)

In [24]:
len(itemlist)

10876

In [30]:
#suwanidf = pd.DataFrame(TRAIN_DATA, columns = ['Text','Annotation'])

In [20]:
#suwanidf = pd.DataFrame(annotations.items(), columns = ['name','annotations'] )

In [31]:
#suwanidf.to_csv(r"D:\FYP\Annotation JSON\Inception\SuwaniALLfromtraininglist.csv")